In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import cos, sin, arcsin, sqrt
from math import radians
from datetime import date
import holidays
from sklearn.cluster import KMeans
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

## Train, test and validation split

In [2]:
df_train = pd.read_csv("data/task_2/pre_task2_2014_2018.csv", index_col=0)
date_train = df_train['start_date']
df_train = df_train.drop(["start_date"], axis=1)

df_valid = pd.read_csv("data/task_2/pre_task2_2019.csv", index_col=0)
date_valid = df_valid['start_date']
df_valid = df_valid.drop(["start_date"], axis=1)

df_test = pd.read_csv("data/task_2/pre_task2_2022.csv", index_col=0)
date_test = df_test['start_date']
df_test = df_test.drop(["start_date"], axis=1)

def df_split(train, valid, test):
    X_train = train.copy()
    y_train = X_train['count']
    X_train = X_train.drop(["count", 'duration_sec'], axis=1)
    
    X_valid = valid.copy()
    y_valid = X_valid['count']
    X_valid = X_valid.drop(["count", 'duration_sec'], axis=1)
    
    X_test = test.copy()
    y_test = X_test['count']
    X_test = X_test.drop(["count", 'duration_sec'], axis=1)
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

X_train, y_train, X_valid, y_valid, X_test, y_test = df_split(df_train, df_valid, df_test)


## Modelling

In the following block we have made functions for 4 different models:
- RandomForrestRegressor
- GradientBoostingRegressor
- TensorFlow
- CatBoost

In [3]:
def random_forrest_regressor(train, valid, test):
    X_train, y_train, X_valid, y_valid, X_test, y_test = df_split(train, valid, test)

    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)

    res = X_valid.copy()
    res["actual"] = y_valid
    res["pred"] = y_pred
    
    #res.to_csv("results/pred_random_forrest_regressor_all.csv")
    return model, res

def gradient_boosting_regression(train, valid, test):
    X_train, y_train, X_valid, y_valid, X_test, y_test = df_split(train, valid, test)

    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)

    res = X_valid.copy()
    res["actual"] = y_valid
    res["pred"] = y_pred
    
    #res.to_csv("results/pred_gradient_boosting_regressor_all.csv")
    return model, res

def tensor_flow(train, valid, test):
    
    def model_prep(train, valid, test):
        df_train = train.copy()
        df_valid = valid.copy()
        df_test = test.copy()
        
        # One hot encoding of Boolean variables
        encoder = OneHotEncoder()
        encoded = pd.DataFrame(encoder.fit_transform(df_train[['is_holiday', 'is_weekend']]).toarray(), columns=encoder.get_feature_names_out(['is_holiday', 'is_weekend']))
        df_train = df_train.drop(columns=['is_holiday', 'is_weekend'])
        df_train = df_train.join(encoded)
        encoded = pd.DataFrame(encoder.transform(df_valid[['is_holiday', 'is_weekend']]).toarray(), columns=encoder.get_feature_names_out(['is_holiday', 'is_weekend']))
        df_valid = df_valid.drop(columns=['is_holiday', 'is_weekend'])
        df_valid = df_valid.join(encoded)
        encoded = pd.DataFrame(encoder.transform(df_test[['is_holiday', 'is_weekend']]).toarray(), columns=encoder.get_feature_names_out(['is_holiday', 'is_weekend']))
        df_test = df_test.drop(columns=['is_holiday', 'is_weekend'])
        df_test = df_test.join(encoded)
        
        # Standard scaler for continuous variables
        scaler = StandardScaler()
        df_train[['duration_sec', 'mean_temperature', 'total_precipitation']] = scaler.fit_transform(df_train[['duration_sec', 'mean_temperature', 'total_precipitation']])
        df_valid[['duration_sec', 'mean_temperature', 'total_precipitation']] = scaler.transform(df_valid[['duration_sec', 'mean_temperature', 'total_precipitation']])
        df_test[['duration_sec', 'mean_temperature', 'total_precipitation']] = scaler.transform(df_test[['duration_sec', 'mean_temperature', 'total_precipitation']])
        
        # Minmax scaler for station cluster ids
        scaler = MinMaxScaler()
        df_train[['start_station_cluster', 'end_station_cluster']] = scaler.fit_transform(df_train[['start_station_cluster', 'end_station_cluster']])
        df_valid[['start_station_cluster', 'end_station_cluster']] = scaler.transform(df_valid[['start_station_cluster', 'end_station_cluster']])
        df_test[['start_station_cluster', 'end_station_cluster']] = scaler.transform(df_test[['start_station_cluster', 'end_station_cluster']])
        
        return df_train, df_valid, df_test
    
    train, valid, test = model_prep(train, valid, test)
    X_train, y_train, X_valid, y_valid, X_test, y_test = df_split(train, valid, test)
    
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_valid, y_valid))

    y_pred = model.predict(X_test)
    
    res = X_test.copy()
    res["actual"] = y_test
    res["pred"] = y_pred
    
    #res.to_csv("results/pred_tensorflow_all.csv")
    return model, res

def cat_boost(train, valid, test):
    X_train, y_train, X_valid, y_valid, X_test, y_test = df_split(train, valid, test)
    
    model = CatBoostRegressor()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_valid)
    
    res = X_valid.copy()
    res["actual"] = y_valid
    res["pred"] = y_pred
    
    #res.to_csv("results/pred_cat_boost_all.csv")
    return model, res

rfr_mod, rfr_res = random_forrest_regressor(df_train, df_valid, df_test)
gbr_mod, gbr_res = gradient_boosting_regression(df_train, df_valid, df_test)
tsf_mod, tsf_res = tensor_flow(df_train, df_valid, df_test)
cbt_mod, cbt_res = cat_boost(df_train, df_valid, df_test)

Epoch 1/20
12440/12440 [==============================] - 5s 418us/step - loss: 3860.4878 - val_loss: 4424.5840
Epoch 2/20
12440/12440 [==============================] - 5s 413us/step - loss: 3813.3074 - val_loss: 4394.8242
Epoch 3/20
12440/12440 [==============================] - 5s 417us/step - loss: 3771.2366 - val_loss: 4366.7407
Epoch 4/20
12440/12440 [==============================] - 5s 419us/step - loss: 3720.9177 - val_loss: 4263.5435
Epoch 5/20
12440/12440 [==============================] - 5s 416us/step - loss: 3626.9675 - val_loss: 4126.1440
Epoch 6/20
12440/12440 [==============================] - 5s 415us/step - loss: 3536.5811 - val_loss: 4067.0356
Epoch 7/20
12440/12440 [==============================] - 5s 411us/step - loss: 3488.1931 - val_loss: 3992.3518
Epoch 8/20
12440/12440 [==============================] - 5s 413us/step - loss: 3448.6890 - val_loss: 4000.2124
Epoch 9/20
12440/12440 [==============================] - 5s 426us/step - loss: 3405.3291 - val_loss: 39

## Evaluating the models

In [4]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.metrics import r2_score

def evaluate(df):
    actual_values = df['actual']
    predicted_values = df['pred']
    
    mse = mean_squared_error(actual_values, predicted_values)
    rmse = np.sqrt(mse)
    correlation_coefficient, p_value = pearsonr(actual_values, predicted_values)
    r2 = r2_score(actual_values, predicted_values)
    
    return rmse, mse, correlation_coefficient, r2

def compare_results(all_results):    
    results = []
    for name, df in all_results.items():
        rmse, mse, correlation_coefficient, r2 = evaluate(df)
        result = {
            'Model': name,
            'RMSE': rmse,
            'MSE': mse,
            'Correlation Coefficient': correlation_coefficient,
            'R-squared': r2
        }
        results.append(result)
    
    results_df = pd.DataFrame(results)
    #results_df.to_csv("results/evaluations.csv")
    return results_df

all_results = {
    "Random Forrest Regressor": rfr_res,
    "GradientBoosting Regressor": gbr_res,
    "TensorFlow": tsf_res,
    "CatBoost": cbt_res
}

print(compare_results(all_results))

                        Model       RMSE          MSE   
0    Random Forrest Regressor  30.624997   937.890438  \
1  GradientBoosting Regressor  59.287515  3515.009493   
2                  TensorFlow  84.764862  7185.081801   
3                    CatBoost  37.118327  1377.770164   

   Correlation Coefficient  R-squared  
0                 0.901788   0.791894  
1                 0.519801   0.220066  
2                 0.461377   0.162625  
3                 0.859596   0.694291  


## Hyperparametertuning (RandomForrestRegressor)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

def hpt_random_forrest(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
    }
    
    rf = RandomForestRegressor(random_state=42)
    
    scorer = make_scorer(mean_squared_error, greater_is_better=False)
    
    random_search = RandomizedSearchCV(
        rf, param_distributions=param_grid, n_iter=5, scoring=scorer, cv=5, random_state=42
    )

    random_search.fit(X_train, y_train)
    
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    best_score = random_search.best_score_
    
    print("Best Hyperparameters:", best_params)
    print("Best Score:", best_score)
    
    predictions = best_model.predict(X_test)

    res = X_test.copy()
    res["actual"] = y_test
    res["pred"] = predictions
    
    return best_model, res

best_model, predictions = hpt_random_forrest(X_train, y_train, X_valid, y_valid)

/Users/fabian/Repos/datamining/data_mining_project/.venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/fabian/Repos/datamining/data_mining_project/.venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/fabian/Repos/datamining/data_mining_project/.venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep th

Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': None}
Best Score: -419.3617432098575


In [9]:
X_train_combined = pd.concat([X_train, X_valid])
y_train_combined = pd.concat([y_train, y_valid])

best_model.fit(X_train_combined, y_train_combined)
new_predictions = best_model.predict(X_test)

res = X_test.copy()
res["actual"] = y_test
res["pred"] = new_predictions

hpt_comparison = {
    "Hpt predictions" : predictions, 
    "Best model new training": res,
    "Default parameters": rfr_res
}

print(compare_results(hpt_comparison))

/Users/fabian/Repos/datamining/data_mining_project/.venv/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


                     Model       RMSE          MSE  Correlation Coefficient   
0          Hpt predictions  30.028352   901.701926                 0.907263  \
1  Best model new training  51.010210  2602.041503                 0.934137   
2       Default parameters  30.624997   937.890438                 0.901788   

   R-squared  
0   0.799924  
1   0.696749  
2   0.791894  


### Baseline

In [13]:
def baseline(X_train, y_train, X_test, y_test):
    baseline = X_test.copy()
    baseline["count"] = y_test
    baseline = baseline[["start_station_cluster", "end_station_cluster", "count"]]

    traning_df = X_train.copy()
    traning_df["count"] = y_train
    traning_df = traning_df[["start_station_cluster", "end_station_cluster", "count"]]
    
    mean_df = traning_df.groupby(["start_station_cluster", "end_station_cluster"]).mean()[["count"]]
    
    merged_df = pd.merge(baseline, mean_df, left_on=['start_station_cluster', 'end_station_cluster'], right_on=['start_station_cluster', 'end_station_cluster'])
    merged_df = merged_df.rename(columns={'count_x': 'actual', 'count_y': 'pred'})
    
    return merged_df

hpt_comparison["Baseline"] = baseline(X_train, y_train, X_test, y_test)
print(compare_results(hpt_comparison))

                     Model       RMSE          MSE  Correlation Coefficient   
0          Hpt predictions  30.028352   901.701926                 0.907263  \
1  Best model new training  51.010210  2602.041503                 0.934137   
2       Default parameters  30.624997   937.890438                 0.901788   
3                 Baseline  62.826382  3947.154302                 0.920783   

   R-squared  
0   0.799924  
1   0.696749  
2   0.791894  
3   0.621416  
